# 01. Data Split

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

In [45]:
rnd_nbr = 42
train_frac = 0.95

## Read data

In [46]:
db = pd.read_excel("../data/raw/archivo_combinado.xlsx")

In [47]:
# Split compund columns
db[['FAM_Sample_info_id', 'FAM_Sample_info_rel']] = db['FAM_Sample_info'].str.split(' ', n = 1, expand = True)
db[['VIC_Sample_info_id', 'VIC_Sample_info_rel']] = db['VIC_Sample_info'].str.split(' ', n = 1, expand = True)
db.drop(['FAM_Sample_info', 'VIC_Sample_info'], axis = 1, inplace = True)

In [48]:
db

,Shared_Markers,k0,k1,k2,Share_allele,Value,True_positive,Marcadores,Software,Datos,Kinship,File,Hoja,Sheet,Value_range,FAM_Sample_info_id,FAM_Sample_info_rel,VIC_Sample_info_id,VIC_Sample_info_rel
0,15,0,4,11,26,236354.0,False,15,BlindSeach,Simulados,Paternidad,15_BlindSeach_Simulados_Paternidad_allele,ipi-1,1:749,100000-999999,012102,F,065402,F
1,15,0,7,8,23,184374.0,False,15,BlindSeach,Simulados,Paternidad,15_BlindSeach_Simulados_Paternidad_allele,ipi-1,1:749,100000-999999,023802,S,099602,F
2,15,0,6,9,24,83900.6,False,15,BlindSeach,Simulados,Paternidad,15_BlindSeach_Simulados_Paternidad_allele,ipi-1,1:749,10000-99999,047302,S,062402,F
3,15,0,10,5,20,41143.0,False,15,BlindSeach,Simulados,Paternidad,15_BlindSeach_Simulados_Paternidad_allele,ipi-1,1:749,10000-99999,116102,F,116702,F
4,15,0,8,7,22,30327.8,False,15,BlindSeach,Simulados,Paternidad,15_BlindSeach_Simulados_Paternidad_allele,ipi-1,1:749,10000-99999,122502,F,083702,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
879331,10,5,3,7,17,2.0,True,15,ScreenMatch,Simulados,Hermandad,15_ScreenMatch_Simulados_Hermandad_allele,Process_lisib-375,375:375,0-99,149701,U,149701,S
879332,10,5,4,6,16,1.0,False,15,ScreenMatch,Simulados,Hermandad,15_ScreenMatch_Simulados_Hermandad_allele,Process_lisib-375,375:375,0-99,149701,U,109202,U
879333,12,3,6,6,18,7.0,True,15,ScreenMatch,Simulados,Hermandad,15_ScreenMatch_Simulados_Hermandad_allele,Process_lisib-375,375:375,0-99,149801,U,149801,S
879334,11,4,5,6,17,3.0,False,15,ScreenMatch,Simulados,Hermandad,15_ScreenMatch_Simulados_Hermandad_allele,Process_lisib-375,375:375,0-99,150002,U,150002,S


In [49]:
uniqueness_features = ["FAM_Sample_info_id", "FAM_Sample_info_rel", "VIC_Sample_info_id", "VIC_Sample_info_rel", "Software", "Datos"]
db.drop_duplicates(subset =  uniqueness_features, inplace = True)

In [50]:
# Guardar los datos simulados y reales en dataframes distintos...
db_simulados = db[db.Datos == "Simulados"]
db_real = db[db.Datos == "Real"]

In [51]:
sim_samples = len(db_simulados)
sim_samples_train = int(sim_samples * train_frac)

In [52]:
# Revolver los datos
db_simulados = db_simulados.sample(frac = 1, random_state = rnd_nbr, ignore_index = True)

db_train = db_simulados.iloc[:sim_samples_train, :]
db_test = db_simulados.iloc[sim_samples_train:, :]

## Balanceo de Clases con Undersampling

In [53]:
db_train_true = db_train[db_train.True_positive == True]
db_train_false = db_train[db_train.True_positive == False].sample(n = len(db_train_true))

In [54]:
db_train = pd.concat([db_train_true, db_train_false], axis=0, ignore_index = True)
db_train = db_train.sample(frac = 1, random_state = rnd_nbr, ignore_index = True)

### Save dataframes as csv

In [55]:
base_path = '../data/processed/balanced/'

db_real.to_csv(base_path + 'real.csv', index = False)
db_train.to_csv(base_path + 'train.csv', index = False)
db_test.to_csv(base_path + 'test.csv', index = False)